<a href="https://colab.research.google.com/github/bolof2000/daily-checkin/blob/master/concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  OWASP TOP 10 
# What is Broken Object Level Authorisation?
Broken Object Level Authorisation all starts with an object. Objects should be looked at in the context of "Object Oriented Programming", what I mean with that is objects are the things you think about first in designing a program and they are also the units of code that are eventually derived from the process. This can be anything, ranging from an account to an invoice to a credit note and everything in between. Usually these objects are marked with an identifier because we need to address these objects directly and this is where issue can arise because we need to always check if the user is allowed to access that object. This might seem simple at first but i can assure you it's not, more about that later in the "testing" section of this article.

A broken Object issue occurs when the server does not properly check if the currently logged in user or if a logged out user can read, update or delete an object to which they do not have the rights.

Two main types of Broken Object Level Authorization
There are two types of broken authorisation on object, these can occur because either a userID is passed on to the server or an objectID, we will look into both.

Based on user ID
Sometimes a userID is passed on to the server, this can happen when we request all the resources for a certain user for example:

<https://google.com/get_users_search_history?userID=1234>
If we replace the userID with someone else's userID, we should not be able to get the search history of other users but when a Broken Object Level Authorization issue arises, we can view the search history of other users.

This issue is simple to solve as a developer. We simply need to check if the currently logged in user is allowed to access those objects. In this example we need to check if the userID from the GET parameter is the same as the userID of the object's owner.



# What is Broken User Authentication?
Broken User Authentication can manifest in several issues. Whenever we come across an API endpoint that handles authentication we need to be extra careful since these endpoints will often determine how a user can flow through the application and what data they see. Whenever one of the following conditions is true, we can speak of a "Broken User Authentication

If your API allows for credential stuffing, This is an attack where a hacker will try to brute force credentials by using a list with known account names and password from other websites. This works because people often re-use their old usernames and passwords so if their credentials leak one time, a lot of accounts are vulnerable. Attackers will take these old credentials and try them on our API endpoints rapidly to see if any of the accounts work on our application. Rate limiting should exits.

If the API endoint does not verify the request with a captcha when needed

When the application allows for weak passwords. The best policy is to use a passphrase instead of a password but anything is better than a weak password.

When the endpoint uses GET parameters (parameters in the URL) to send sensitive data such as passwords or tokens. This could allow for a MiTM attack.

When our endpoint issues a token such as JWT but does not validate it.

When we use JWT, we should be aware that we should not accept unsigned or weakly signed tokens.

When the endpoint does not validate the expiration date of authentication tokens such as session tokens or JWT

When the endpoint does not encrypt or hash the password or uses weak encryption algorithms.

When it uses weak encryption keys that are easy to guess (Like a birthday) or easy to crack (like md5)


An API is only supposed to return the required data to the front-end clients but sometimes developers will make a mistake or take the easy route and implement generic API's that return all data to the client. When these API's return too much data, we can speak of Excessive Data Exposure.

Example Attack Scenarios
A simple example we can give is an application which makes a call to grab the credit card details. The user does not see the CCV because it will be filtered out by the front-end client but the API still returns too much data.

Example:

GET /api/v1/cards?id=0
 
[
  {
    "CVV": "677", 
    "creditCard": "1234567901234", 
    "id": 0, 
    "user": "API", 
    "validUntil": "1992"
  }
]
 
As you can see here, we made the call to grab the credit card details and while the end user might not be able to see the CVV but since the API returns it, we are speaking of Excessive Data Expsoure.





# What is a lack of rate limiting?
Whenever an API is served a request it will have to respond, to generate this response the API requires resources (CPU, RAM, network and at times even disk space) but how much are required highly depends on the task at hand. The more logic processing happens or data is returned, the more resources are taken up by a single call and this can stack on quick. If we do not rate limit our API endpoints. This issue is made even worse by the fact that most API's reside on shared hosts which means they are all fighting for the same resources which could mean the attacker is disabling a secondary unrelated API by consuming all the resources.

Example Attack Scenarios
There are simple examples of attacks related to lack of rate limiting on endpoint but those are easy enough, a somewhat deeper attack could be a user who discovers the endpoint to create a file which does have rate limiting and an endpoint to copy a file does not have rate limiting. At first this might seem hard to abuse but if we create a document on the system that has a large file size and then copy it over, we might trigger the server to run out of resources.

Example:

POST /createDocument.php
 
[
  {
    "Name": "677", 
    "Content": "AAAAA...AAAA"
		"fileSize":"21343243242343kb"
  }
]
 
With a response of the ID:
 
id=12
 
If we try to trigger this call multiple times we will notice rate limiting on the endpoint.
 
GET /cloneDocument.php?id=12
 
But there might be a GET call which is not rate limited and by triggering it multiple times we might consume all of the server's resources.
Let's add another example to make things more clear. We might be trying to recall the last 100 posts to a blog with the following URL

GET /api/v1/posts?limit=100

By executing this request with a parameter of limit=99999 we might trigger a lack of resources as well and this is also counted as lack of endpoint rate limiting.

Preventive measures against Excessive Data Exposure
Using a docker container can be very handy as it has built in flags such as -m to determine exactly how much memory every container may consumer. This also allows for a much easier seperation of different APIs.

We should make sure the client can only make a certain amount of request over a certain period, If we do this however we have to make sure that the client is notified when a rate limit is triggered or about to be triggered. This message should contain the amount of remaining requests before the limit is hit and/or the remaining time of the rate limit

We need to verify on the client and server side that the request body and response are not too big. This is especially true for endpoints that return an amount of records specified by the client. The client can manipulate the amount of requests and cause a severe issue to occur if they request too many records at a time.

Every data type that the endpoint accepts should have an upper limit, for example integers should be limited to 5999. This en


# New Section
# What is Broken Function Level Authorization?
From the summary you might have been able to gather this vulnerability can be quite complex and varied. Humans are predictable in nature and they tend to follow several patterns when it comes to hosting endpoints, for example it usually occurs that higher priviledge endpoints are hosted on the same relative path, which makes it easier for the users to guess these endpoints but that alone does not make the vulnerability though. We have to be able to access to endpoints as a lower priviledged users as well of course. This access can occur by executing the provided HTTP method (for example POST) but it can also occur on other methods that the developers left enabled (un)intentionally (for example DELETE). These issues are often made worse by how predictable the endpoints are (for example /books?id=0 might indicate the presence of an endpoint /books/all which may print out books the user should not be able to access.).

Example Attack Scenarios
In our first example we will be looking at a target which contains two types of accounts: Franchise holder and a company looking to exploit the franchise. Both log into the same URL but the Franchise holder type of account is much more expensive. However by going to the account settings and saving, we notice a hidden parameter "Account type" and we change it from "exploitant" to "franchise_holder". This allows users to buy a lower priced type of account and change it to higher priced account type themselves later on.

Example:

POST /api/saveSettings.php
 
[
  {
    "Name": "677", 
    "...": "AAAAA...AAAA"
		"AccountType":"exploitant"
  }
]
 
We change this body to 
 
POST /api/saveSettings.php
 
[
  {
    "Name": "677", 
    "...": "AAAAA...AAAA"
		"AccountType":"franchise_holder"
  }
]
and by sending this request, the users can upgrade their own accounts to a more expensive account type.
A second example we can add is where users try to view documents however they can only view their own. By exporting their own documents though, the user is presented a URL /api/documents/export. This URL does return the proper documents and only the only belonging to the user but by simply guessing, the user discovers a URL that returns all the books on /api/documents/export/all.

GET /api/documents
 
Will return the following response
 
[
  {
    "id": "1", 
    "content": "AAAAA...AAAA"
		"creationDate":"10-10-10"
  },
  {
    "id": "4", 
    "content": "AAAAA...AAAA"
		"creationDate":"10-10-10"
  }
]
 
Since this user can only see their own documents, they can only see those 2, however if they export the documents, the user notices they can change the path and export all the documents.
 
GET /api/documents/export/
 
will return the same as before, however the following:
 
GET /api/documents/export/all
 
exports all the documents.
Preventive measures against Excessive Data Exposure
It's always better to seperate the authentication and authorization module from the main code and make it easy to implement where needed. By default all access should be denied to make it safer, we can also work on blocking the traffic we do not want but this would make it easier to miss certain paths of attack.




# What is Mass Assignment?
Applications these days often rely an objects (For example user, product, ...) and these objects have properties (for example product.stock). As a user, we have the authorization to edit and view specific properties of the objects but we might also be limited and not able to edit or view some specific properties (For example user can view product.stock but user should not be able to edit product.stock). These properties are then matched to parameters on the front-end and if these conversion happen automatically, they might convert parameters to properties the attacker should not be able to access (For example, the user should never be able to edit product.title but the front-end might convert a paramter "title" to product.title if the user sends a PUT request).

Here are some more examples of properties the user should not be able to edit:

Account.AccountType or Account.discountsEnable. These are properties that relate to permissions.

Account.wallet This property should never be editable be the attacker

product.title These are internal properties the user should never be able to edit

Example Attack Scenarios
These attack scenario's come from my personal experience as a bug bounty hunter. This happens to be my favourite issue types because it's really to miss them and you can not automate the search easily due to the required logic knowledge.

The first example is from an application that allows you to make an appointment which last 1 hour. In the UI I am able to see timeslots that last 1 hour and i can select one of them.


I was staring at the request and did not notice it at first, it took me a good hour to realise it:

{
"startDate":"29/04/2022 11:00",
"endDate":"29/04/2022 12:00",
"userID":"123",
"consultantID":"123"
}
If we change the start or end date we can fully book the agenda of the consultant for years if we wish. This is very easy to miss which is why i like this issue type so much!

{
"startDate":"29/04/2022 11:00",
"endDate":"29/04/2099 12:00",
"userID":"123",
"consultantID":"123"
}
A second example is that i love to look at my account settings when i am hacking to see if i can't find any properties in the api responses that I should not be able to edit to make myself admin for example but this is about a much more subtle bug.

Request:
{
"endDate":"29/04/2099 12:00",
"userID":"123",
"consultantID":"123"
}
 
Response from API:
 
{
"AccountType":"airport",
"endDate":"29/04/2099 12:00",
"userID":"123",
"consultantID":"123"
}
There are two account types in here, these account types can not be found in the requests themselves but only in the respones however if i copy over that parameter, i might be able to change it as the API might automatically map the object.

Request:
{
"endDate":"29/04/2099 12:00",
"userID":"123",
"consultantID":"123"
"AccountType":"airline",
}
 
Response from API:
 
{
"AccountType":"airline",
"endDate":"29/04/2099 12:00",
"userID":"123",
"consultantID":"123"
}



# What is Security Misconfiguration?
There are several factors that might indicate a Security Misconfiguration. We should be very careful with handling configurations because if the correct security measures are not in place to protect our APIs, an attacker might be able to take over the full infrastructure.

We have to ensure all our systems are always up to date to avoid old exploits working on our systems. Following up on this, if our systems are up to date, we have to disable any unused functions like http PUT calls.

All our data should always be transported over a TLS channel to avoid that an attacker can perform a MitM attack.

Make sure all your security headers such as CSP are working correctly and configured wherever needed. With CSP enabled we should also run a CORS policy and configure it properly or you might open yourself up to Security Misconfiguration vulnerabilities.

Furthermore as a last point we can claim that security misconfigurations happen when the end user is able to see error messages or warnings. These should only be logged and viewed internally.

How to detect Security Misconfiguration?
First of all, it is really important that we look get an overview of our entire application architecutre. We need to gain visibility by creating a mindmap or a schema.


Make sure you include everything in here, from printers to smart thermostats and mobile phones. These days you should even include watches if your company allows smartwatches to connect to the network.

Now that we have everything mapped, we need to scan it for security misconfigurations, we can do this manually or with a scanner. To do this manually, go over all the assets, workflows and everything else you gathered and confirm they are configured correctly. You are looking for things like passwords sent over plain text or unencrypted communication on server x which would indicate Security Misconfiguration.

Hybrid-cloud environments and Security Misconfiguration

A lot of people are against hybrid environments but it's been given a bad rep due to the misconfigurations that are often present on these environments with improper network protection and execution. This can often lead to vulnerabilities, we will go over a few of them.

First of all it is really important to encrypt all the traffic that moves over the network to give attackers who perform a man-in-the-middle attack no chance to eavesdrop on the data being transmitted.

If we want to set up a complex environment like this, we have to perform a really thorough risk analysis as well to make but this is often overlooked or not executed due to budget constraints. It's really important to have rigerious risk assesment in place and evaluate it from time to time.

What is also vital is proper security management is implemented. Sometimes shortcuts will be built in for testing purposes that make it to the production environment, this should never happen but also the intended authentication methods should be strong and should not allow for attacks.

Our different cloud envirnoments need to be coordinated carefully and we need to assure the fall within compliance rules. This is extra important in hybrid-cloud environments where two clusters might need to communicate at any time.





# What is Injection?
API's with the following properties are open to injection flaws:

When we don't sanitize the input from the front-end we are opening ourselves to a world of problems, this would allow the user to input anything which could intervene with later processes.

The same goes for validation and verification of the API's request, these have to be done before the data enters any kind of processes, this is where it could start causing problems for example on the login calls, make sure the API is validating, verifying and sanitizing any input.

Make sure that you pay attention to not just the input from the user directly, this can also come from 3rd party services or file uploads for example.

Take into account there might be processes such a batch jobs running that could trip over the data.

Example Attack Scenarios
I can also draw from experience on this vulnerability type as i have and reported on it often. The form i prefer is SQL injections via the import feature and OS command injection from an unexpected source. Let's start with the SQL injection.

I found this issue while doing bug bounties on a private program and it happend because the developers did sanitise all the direct input very well, except they did not think to include the import functionality because the base of the application was built a year prior to building the import functionality.

Upload.csv looked like this: 
name,adress,email,phone
',',','
 
And while uploading i selected the comma as field seperator, this displayed a SQL error and from there on i dug in deeper.
 
The error was:
Expects paremeter 1 to be string, null given in /var/www/html/import.php
This made me close the query and start a new one of my own
name,adress,email,phone
';select * from users;--,',','
This made the application dump the entire uses table in an error message, that was enough for me to report this issue and collect my bounty.

The second example i have is a little bit less complicated, i noticed a parameter litteraly called "osParam" which seemed to have some flags in it, i rushed to start up burp suite intruder with a list of command injections i had prepared before and had a hit on the 9th request that burp suite made. The command seperate was a newline character '\n' and my ping command delayed the response.

index.php?osParam=\\nping -c 10 127.0.0.1 
So i quickly tried a whoami, reported the result and awaited approval which cames 2 days later.

Preventive measures against Injections?
We need to treat any input as being compromised and we should filter, validate en verify every input to our API through all ways, this includes third party inputs or non-direct inputs such as importing files.

We have to make sure to create 1 system that will be able to perform these steps and that we implement and use that same system across all of our endpoints.

Prefereably use a well known library that has been tested instead of creating your own.

We have to take care in filtering special characters, often the language we use has a specific syntax and way to handle these special characters and it's adviced to implement that syntax.

Whener multiple records are being selected, limit the number of records per query to avoid mass disclosure

Preferably use a specification that specifies how the API works such as OpenAPI and that you only allow requests that match the filters, verification and validation rules

Define what the API can expect for all the string endpoints and in terms of variable types.

An example in php which is badly implemented and leaves the app open for things such as XSS 
		$id = $row['id'];
    $title = $row['title'];
    $des = $row['description'];
    $time = $row['date'];
This is how it's supposed to be done
		$id = htmlentities($row['id']);
    $title = htmlentities($row['title']);
    $des = htmlentities($row['description']);
    $time = htmlentities($row['date']);




# What is Improper Assets Management?
We should always wonder for every API if all the current endpoint should even be aviable and if we maybe can't do with only allowing the whole API to communicate internally. To aid us we can ask ourself if this API even needs to be in production at all and who should have access to the API. If the specific API does needs to be in production, we can also ask ourselves if we are not using an outdated version of the API, if there is sensitive data exposed or how the data flows throughout the application and APIs.

A lack of documentation is a problem that plagues many companies and this only makes things worse because an undocumented API that does not send out traffic can remain undetected for a long time.

To ease the removal of older APIs we might consider a retirement plan for APIs that are no longer needed. Another tool we have at our disposal are inventory management systems which index every API and it's version, this can be used to perform a regular inspection of the API plan.

As long as these older APIs and their libraries remain unpatched, the system will be vulnerable to Improper Assets Management.

Example Attack Scenarios
Our first example will be a scenario that i encountered in production. I was scanning the APIs with a directory brute forcer to see if i could not find anything and unsurprisngly i could not find a thing. I noticed however that i was working on version 2 of the API from the URL:

<http://WEBSITE:5000/api/v2/resources/books/all>
After replacing the v2 in the URL to v1 and restarting my directory brute forcing attempts i did find an admin URL

<http://WEBSITE:5000/api/v1/admin>
The admin interface was protected in that it could only be accessed from the internal network but this was easily fooled by adding an "x-forwarded-from" header and setting it to 127.0.0.1.

Our second example comes from a third party vendor. For a new feature to work, the target had to include certain libraries which were using outdated APIs but due to imperfect asset management, our target did not have an overview of these APIs existance. These endpoints ultimately led to more functionality being exposed which had an RCE in it.

Preventive measures against Improper Assets Management?
We need to ensure we have a proper inventory management system set up that includes all the API endpoints with what is special about them, their environment and from which networks they are reachable. Also inventory third party integrations for these APIs, so we can have an overview of critical infrastructre that should be easy to consult by the people who need it.

We all know software is becoming more and more dynamic but with that also comes that we will have to review our inventory system on a regular basis and evaluate it.

Some interesting things to document could be:

Errors

Authentication flows

Rate limiting

redirects

CORS policy triggers

These days we can automatically generate the documentation from adopting an API specification such  such as Open API. This will make it much harder to miss a rogue API, however still not impossible. This documentation should be available to users of the API.


# What is Insufficient Logging & Monitoring?
The title already says a lot but this vulnerability is a bit more complext than it was at first sight, of course the API is vulnerable if it does not create any log entries or when the log level is set incorrectly but we should not neg lect to also check whether or not the contents of the log messages are what is expected and that they contain enough detail.


A second aspect that is a bit harder to control is the log integrity, for example a malicious user might insert a special character that when written to the log breaks it. This is known as log injection and it could nullify all the logging efforts made.

When we finally have a clear plan of what to log, when and on what environment we need to ensure we are monitoring these logs, failure to do so would result in a massive decrease in the efficiency of our logging efforts.




The finalize() method is a protected and non-static method of the java.lang.Object class and is also sometimes called the finalizer. The finalize() method is invoked by the JVM when an object is garbage collected. A user can override the finalize() in his class to dispose of system resources or to perform cleanup tasks.

**Generics **
Generics help with the following use-cases

Enforcing stronger type checking at compile time for greater type-safety

Elimination of casts

Implementation of generic algorithms



A mutex allows only a single thread to access a resource. It forces competing threads to serialize their access for the requested resource.

Semaphore, on the other hand, is used for limiting access to a collection of resources. Think of semaphore as having a limited number of permits to give out. If a semaphore has given out all the permits it has, then any new thread that comes along requesting for a permit will be blocked, till an earlier thread with a permit returns it to the semaphor



class Solution:
    def findPeakElement(self, nums: List[int]) -> int:
        
        left =0
        right = len(nums)-1
        while left < right:
            mid = (left+right)//2
            if nums[mid] < nums[mid+1]:
                left = mid+1
            else:
                right = mid
        
        return left 
        

  class Solution:
    def findMin(self, nums: List[int]) -> int:
        
        right = len(nums)-1
        left =0
        
        while left+1 < right:
            mid = (left+right)//2
            
            if nums[mid] < nums[right]:
                right = mid
            else:
                left = mid
        return min(nums[left],nums[right]

class Solution:
    def findAnagrams(self, s: str, p: str) -> List[int]:
        ns, np = len(s), len(p)
        output = []
        if ns < np:
            return []
        p_count, s_count = [0] * 26, [0] * 26 
        for ch in p:
            p_count[ord(ch) - ord('a')] += 1
        
        for i in range(ns):
            s_count[ord(s[i]) - ord('a')] += 1
            
            if i >= np:
                s_count[ord(s[i - np]) - ord('a')] -= 1
            if p_count == s_count:
                output.append(i - np + 1)
        
        return output

Aysn Programming 

Completable Future -
program for aysnc programming

Using Async programming you can write non blocking code where concurrently yhou canrun N no of tasks in a seperate thread without blocking the main thread 
when the task is complete,it notifies to the main thread 


Completable Future - program for aysnc programming

Using Async programming you can write non blocking code where concurrently yhou canrun N no of tasks in a seperate thread without blocking the main thread when the task is complete,it notifies to the main thread

Java Reflection makes it possible to inspect classes, interfaces, fields, and methods at runtime, without knowing the names of the classes, methods etc. at compile time. It is also possible to instantiate new objects, invoke methods and get/set field values using reflection.




When to use it?
Anonymous class is only created for one-time use, we cannot reuse it. Let us consider some scenario

When your only purpose is to override a method.
When you want to implement any functional interface. (We will see it with the help of real-world implementation).

Explain the garbage collection process in Java.

Whenever a new object is created, it is allocated its space in the Eden memory. The Eden memory is limited and fills up fast. Initially, both the survivor spaces are empty. Let's call them survivor space A and B to distinguish between them.

When the garbage collector runs for the first time, it marks alive objects in Eden memory and any unused objects are collected as garbage. The objects surviving the garbage collector run are placed in one of the survivor spaces. For our example let's imagine they are placed in survivor space A.

On the second run of the GC the surviving objects from the Eden memory and the survivor memory A are placed in the unused survivor memory B.

# Multithreading:
Java’s most fundamental construct for thread synchronization is the synchronized keyword. It can be used to restrict access to critical sections one thread at a time.

Each object in Java has an entity associated with it called the monitor lock or just monitor. Think of it as an exclusive lock. Once a thread gets hold of the monitor of an object, it has exclusive access to all the methods marked as synchronized. No other thread will be allowed to invoke a method on the object that is marked as synchronized and will block, till the first thread releases the monitor which is equivalent of the first thread exiting the synchronized method.

Projects-
1. Automated solutions for certifying multiple customers to increase volumes and velocity of testing 
2. Improved Test reporting through report classifications to reduce test results analysis
3. Work with team to develop accessibility testing solutions through development of engines and UI components that abstract three major accessibility tools together via API to increase productivity and improve accuracy of testing across all teams 
4. Develop performance tests solutions to test replications and load distributions for merchant groups which impacts over 20 customers across UI. 
    1. Improve load distributions to multiple servers
    2. Scale to more DB operations to ensure applications fails gracefully under heavy loads
5. Developed code quality toolings



Mocking HTTP requests
Why we mock:
1. Speed 
2. Avoid rate limit - how many requests within some minutes range can this API take?
3. Error Scenarios
4. Deterministic 
5. Avoid modifying state 
6. Testing and TDD